# 每日收盤行情

* 至 [台灣證券交易所/交易資訊/每日收盤行情](https://www.twse.com.tw/zh/page/trading/exchange/MI_INDEX.html)頁面，分類選擇「全部(不含權證、牛熊證、可展廷牛熊證)」
* 找到 csv 檔案的下載網址
* 使用 requests & pandas 套件下載及讀取資料
* 將資料經適當的處理 (轉換格式、刪除等)
* 儲存至資料庫

In [1]:
import requests, sqlite3
from io import StringIO
import pandas as pd

# 輸入擷取日期
date = input('請輸入擷取日期（格式:20200327）：')

# 使用例外判斷擷取資料
url = 'https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date='+date+'&type=ALLBUT0999'

try:
    data = requests.get(url)
except Exception as e:
    print('**WARRN: 擷取失敗')
    print('錯誤訊息', e)
else:
    print(data.text[:300])
    
    # 依換行 \n 符號分割擷取資料（取得每一列資料）
    # 用以判斷哪些列資料是要保留的個股每日收盤資料
    lines = data.text.split('\n')

    # 保留欄位數 > 10 的列資料
    newdata = []
    for line in lines:
        if len(line.split(',"')) > 10:
            newdata.append(line)

    # 轉換 list 型別為 str
    data = '\n'.join(newdata)

    # 移除股票代號前的等號 
    data = data.replace('=', '')

    # 將 str 型號轉為 memory file (string buffer)
    data_io = StringIO(data)

    # 從記憶體中讀取股價資料到 dataframe
    df = pd.read_csv(data_io)

    # 刪除不需要的欄位
    df.drop('漲跌(+/-)', axis=1, inplace=True)
    df.drop('漲跌價差', axis=1, inplace=True)
    df.drop('本益比', axis=1, inplace=True)
    df.drop('Unnamed: 16', axis=1, inplace=True)

    # 移除資料裡的逗號
    df = df.apply(lambda s: s.str.replace(',', ''))

    # 設定索引
    df.set_index(['證券代號','證券名稱'], inplace=True)

    # 將欄位資料由 str 型別轉為數字型別
    df = df.apply(lambda s:pd.to_numeric(s, errors='coerce'))

    # 新增日期欄位
    df['date'] = pd.to_datetime(date, format='%Y%m%d', errors='ignore')

    # 重設索引及更換欄位名稱
    df = df.reset_index()
    df = df.rename(columns={'證券代號':'sid'})
    df.set_index(['sid','date'], inplace=True)

    # 將 dataframe 資料寫入資料庫
    conn = sqlite3.connect('stock1.db')

    # 判斷 price table 是否已存在
    # sqlite會自動維護一個系統表sqlite_master，該表儲存了我們所建立的各個table, view, trigger等等資訊
    query = "select count(*) from sqlite_master where type='table' and name='price'"
    exist = list(conn.execute(query))[0][0]

    # 如果 price table 存在，新增資料，否則，建立表格及新增資料
    if exist:
        df.to_sql('price', conn, if_exists='append')
        print(date, '資料新增完畢!')
    else:
        df.to_sql('price', conn, if_exists='replace')
        print(date, '資料儲存完畢!')

    conn.close()

請輸入擷取日期（格式:20200327）：20200323
"109年03月23日 價格指數(臺灣證券交易所)"
"指數","收盤指數","漲跌(+/-)","漲跌點數","漲跌百分比(%)","特殊處理註記",
"寶島股價指數","10,109.14","-","382.82","-3.65","",
"發行量加權股價指數","8,890.03","-","344.06","-3.73","",
"臺灣公司治理100指數","5,161.28","-","220.30","-4.09","",
"臺灣50指數","7,053.46","-","318.11","-4.32","",
"臺灣50權重上限30%指數","6,926.88","
20200323 資料新增完畢!


# 處理資料擷取錯誤或該日未開盤情況-使用副程式

In [3]:
import requests, sqlite3
from io import StringIO
import pandas as pd

def crawl_price(date):
    url = 'https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date='+date+'&type=ALLBUT0999'

    # 擷取資料，若擷取失敗則返回
    try:
        data = requests.get(url)
    except Exception as e:
        print('日期:', date, '資料擷取失敗!')
        print(e)
        return None

    # 如果資料為空，表示當天未交易，則返回
    if data.text == '':
        return None
    
    # 依換行 \n 符號分割擷取資料（取得每一列資料）
    # 用以判斷哪些列資料是要保留的個股每日收盤資料
    lines = data.text.split('\n')

    # 保留欄位數 > 10 的列資料
    newdata = []
    for line in lines:
        if len(line.split(',"')) > 10:
            newdata.append(line)

    # 轉換 list 型別為 str
    data = '\n'.join(newdata)
  
    # 移除股票代號前的等號 
    data = data.replace('=', '')

    # 將 str 型號轉為 memory file (string buffer)
    data_io = StringIO(data)

    # 從記憶體中讀取股價資料到 dataframe
    df = pd.read_csv(data_io)

    # 刪除不需要的欄位
    df.drop('漲跌(+/-)', axis=1, inplace=True)
    df.drop('漲跌價差', axis=1, inplace=True)
    df.drop('本益比', axis=1, inplace=True)
    df.drop('Unnamed: 16', axis=1, inplace=True)

    # 移除資料裡的逗號
    df = df.apply(lambda s: s.str.replace(',', ''))

    # 設定索引
    df.set_index(['證券代號','證券名稱'], inplace=True)

    # 將欄位資料由 str 型別轉為數字型別
    df = df.apply(lambda s:pd.to_numeric(s, errors='coerce'))

    # 新增日期欄位
    df['date'] = pd.to_datetime(date, format='%Y%m%d', errors='ignore')

    # 重設索引及更換欄位名稱
    df = df.reset_index()
    df = df.rename(columns={'證券代號':'sid'})
    df.set_index(['sid','date'], inplace=True)    
    
    return df


# 輸入擷取日期
date = input('請輸入擷取日期（格式:20200327）：')

data = pd.DataFrame()
data = crawl_price(date)
        
if data is None:
    print('每日收盤行情資料擷取失敗！確認是否日期錯誤，或該日未開盤！')
else:
    # 將 dataframe 資料寫入資料庫
    conn = sqlite3.connect('stock1.db')

    # 判斷 price table 是否已存在
    # sqlite會自動維護一個系統表sqlite_master，該表儲存了我們所建立的各個table, view, trigger等等資訊
    query = "select count(*) from sqlite_master where type='table' and name='price'"
    exist = list(conn.execute(query))[0][0]

    # 如果 price table 存在，新增資料，否則，建立表格及新增資料
    if exist:
        data.to_sql('price', conn, if_exists='append')
        print(date, '資料新增完畢!')
    else:
        data.to_sql('price', conn, if_exists='replace')
        print(date, '資料儲存完畢!')

    conn.close()

請輸入擷取日期（格式:20200327）：20200320
20200320 資料新增完畢!
